In [ ]:
# Requirements: python >= 3.0; Pytorch >= 2.0.0 

# pip install TotalSegmentator 

# Next, please get an academic license for TotalSegmentator (we need it to process the task "appendicular_bones") - should be done in a few seconds! 
# https://backend.totalsegmentator.com/license-academic/
# Original GitHub site containing license information: 
# https://github.com/wasserth/TotalSegmentator#:~:text=Available%20with%20a%20license%20(free%20licenses%20available%20for%20non%2Dcommercial%20usage%20here.%20For%20a%20commercial%20license%20contact%20jakob.wasserthal%40usb.ch)%3A

# Set your license, following email instructions 
# totalseg_set_license -l <your-license-number>

# Download weights of the pre-trained model
# totalseg_download_weights -t total [femur_left, femur_right]
# totalseg_download_weights -t appendicular_bones [patella, tibia, fibula]

In [1]:
import time
import subprocess
import numpy as np
import SimpleITK as sitk 

In [2]:
# Input and output directories
ct_dir = "/mnt/c/users/avery/Desktop/PI201/DICOM/P0000001/ST000001/SE000003"  # Input CT directory 
seg_dir = "/mnt/c/users/avery/Desktop/segmentation_masks"  # Output segmentation masks directory

# Import DICOM
reader = sitk.ImageSeriesReader()
series_ids = reader.GetGDCMSeriesIDs(ct_dir)
if not series_ids:
    raise ValueError(f"No DICOM series found in directory: {ct_dir}")
series_file_names = reader.GetGDCMSeriesFileNames(ct_dir, series_ids[0])
reader.SetFileNames(series_file_names)
image = reader.Execute()

# Extract metadata
ct_dims = image.GetSize()         # Returns a tuple, e.g., (width, height, depth)
ct_spacing = image.GetSpacing()     # Returns voxel spacing, e.g., (spacing_x, spacing_y, spacing_z)
ct_origin = image.GetOrigin()       # Returns the origin of the image

# Calculate voxel volume (mm^3)
voxel_volume = ct_spacing[0] * ct_spacing[1] * ct_spacing[2]

print("Image Dimensions (Width, Height, Depth):", ct_dims)
print("Spacing (mm):", ct_spacing)
print("Origin:", ct_origin)
print("Voxel Volume (mm^3):", voxel_volume)

Image Dimensions (Width, Height, Depth): (512, 512, 251)
Spacing (mm): (0.485, 0.485, 1.0)
Origin: (-147.882, -125.382, 892.5)
Voxel Volume (mm^3): 0.235225


In [3]:
# TotalSegmentator 

# Commands to run segmentations for trained classes 
command_total = f"TotalSegmentator -i {ct_dir} -o {seg_dir} --ta total" # "total"
command_appendicular = f"TotalSegmentator -i {ct_dir} -o {seg_dir} --ta appendicular_bones" # "appendicular_bones"

# Record starting time
start_time = time.time()

# Run first command (femurs)
print("Running segmentation with '--ta total'...")
result_total = subprocess.run(command_total, shell=True, capture_output=True, text=True)
if result_total.stderr:
    print("Errors:")
    print(result_total.stderr)
print("Finished '--ta total' segmentation./n")

# Run second command (patella, tibia, fibula)
print("Running segmentation with '--ta appendicular_bones'...")
result_appendicular = subprocess.run(command_appendicular, shell=True, capture_output=True, text=True)
if result_appendicular.stderr:
    print("Errors:")
    print(result_appendicular.stderr)
print("Finished '--ta appendicular_bones' segmentation./n")

# Display time elapsed
elapsed_time = time.time() - start_time
print(f"Total elapsed time: {elapsed_time:.2f} seconds")



Running segmentation with '--ta total'...
Errors:

  0%|          | 0/8 [00:00<?, ?it/s]/home/avery/torchenv/lib/python3.12/site-packages/torch/nn/modules/conv.py:605: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:84.)
  return F.conv3d(

100%|██████████| 8/8 [00:03<00:00,  2.17it/s]

100%|██████████| 8/8 [00:00<00:00,  9.94it/s]

100%|██████████| 8/8 [00:00<00:00,  9.46it/s]

100%|██████████| 8/8 [00:00<00:00,  9.37it/s]

100%|██████████| 8/8 [00:00<00:00, 11.55it/s]

Finished '--ta total' segmentation./n
Running segmentation with '--ta appendicular_bones'...
Errors:

  0%|          | 0/8 [00:00<?, ?it/s]/home/avery/torchenv/lib/python3.12/site-packages/torch/nn/modules/conv.py:605: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:84.)
  return F.conv3d(

100%|██████████| 8/8 [00:02<00:00,  2.72it/s]

Finished '--ta 

In [4]:
# Smooth edges by resampling with Gaussian filter and thresholding 

def smooth_segmentation_mask(mask_path, ct_spacing, output_path, sigma=2):
    """
    Load a segmentation mask, resample it from TotalSegmentator's default 
    spacing (weights trained on CT images with isotropic 1.5 mm grids;
    its segmentated masks default to 1.5mm isotropic spacing) 
    to target CT spacing; then apply smoothing, re-threshold, and verify
    output spacing equals input CT spacing. 
    
    Main ideas:
      1. Resamples the mask using nearest-neighbor interpolation to preserve binary map
      2. Casts the resampled image to float
      3. Applies low-level Gaussian smoothing (SmoothingRecursiveGaussian) edges
      4. Thresholds the smoothed image to recover a binary mask
    
    Parameters:
      mask_path (str): Path to the input segmentation mask. 
      ct_spacing (array-like): Target CT spacing like [x-spacing, y-spacing, z-spacing]
      output_path (str): Path to save the smoothed segmentation mask.
      sigma (float): Gaussian sigma (default=2). Lower sigma results in less smoothing.
    """
    start = time.time()
    
    # Read the segmentation mask using SimpleITK.
    mask = sitk.ReadImage(mask_path)
    
    # TotalSegmentator output segmentation masks  isotropic masks at 1.5 mm spacing by default.
    target_spacing = tuple(ct_spacing)
    target_size = [int(sz) for sz in np.array(mask.GetSize())]
    
    # Resample using nearest neighbor to preserve the binary mask 
    # Nearest neighbour interpolator recommended by sitk: 
    # https://simpleitk.org/doxygen/v2_4/html/classitk_1_1simple_1_1ResampleImageFilter.html#:~:text=An%20example%20would%20be%20a%20mask%20indicating%20the%20segmentation%20of%20a%20brain%20into%20a%20small%20number%20of%20tissue%20types.%20For%20such%20an%20image%2C%20one%20does%20not%20want%20to%20interpolate%20between%20different%20pixel%20values%2C%20and%20so%20NearestNeighborInterpolateImageFunction%20%3C%20InputImageType%2C%20TCoordRep%20%3E%20would%20be%20a%20better%20choice.
    resampler = sitk.ResampleImageFilter()
    resampler.SetOutputSpacing(target_spacing)
    resampler.SetSize(target_size)
    resampler.SetOutputDirection(mask.GetDirection())
    resampler.SetOutputOrigin(mask.GetOrigin())
    resampler.SetInterpolator(sitk.sitkNearestNeighbor)
    resampled = resampler.Execute(mask)
    
    # Cast the resampled mask to float so that smoothing can produce fractional values
    resampled_float = sitk.Cast(resampled, sitk.sitkFloat32)
    
    # Apply a SmoothingRecursiveGaussian filter with a low sigma 2
    smoothed = sitk.SmoothingRecursiveGaussian(resampled_float, sigma)
    # Threshold the smoothed image back to binary.

    # Since the mask is binary (0/1), set cutoff as 0.5 
    thresholded = sitk.BinaryThreshold(smoothed, lowerThreshold=0.5, upperThreshold=1e9, insideValue=1, outsideValue=0)
    
    # Save the new (smoothed) mask
    sitk.WriteImage(thresholded, output_path)
    elapsed = time.time() - start
    print(f"Smoothing and resampling time for {mask_path}: {elapsed:.2f} seconds")
    
    # --- Post-check: Load output image and print metadata ---
    # Check that segmentation mask metadata is same as CT! 
    output_image = sitk.ReadImage(output_path)
    dims = output_image.GetSize()
    voxel_size = output_image.GetSpacing()
    origin = output_image.GetOrigin()
    print("Output mask dimensions:", dims)
    print("Output mask voxel spacing (mm):", voxel_size)
    print("Output mask origin:", origin)
  

target_ct_spacing = ct_spacing  # mm 
for m in ["femur_left", "femur_right", "patella", "tibia", "fibula"]:
  smooth_segmentation_mask(f"{seg_dir}/{m}.nii.gz", ct_spacing=target_ct_spacing, output_path=f"{seg_dir}/{m}_smoothed.nii.gz")


Smoothing and resampling time for /mnt/c/users/avery/Desktop/segmentation_masks/femur_left.nii.gz: 1.39 seconds
Output mask dimensions: (512, 512, 251)
Output mask voxel spacing (mm): (0.48500001430511475, 0.48500001430511475, 1.0)
Output mask origin: (-147.8820037841797, 122.4530029296875, 892.5)
Smoothing and resampling time for /mnt/c/users/avery/Desktop/segmentation_masks/femur_right.nii.gz: 0.98 seconds
Output mask dimensions: (512, 512, 251)
Output mask voxel spacing (mm): (0.48500001430511475, 0.48500001430511475, 1.0)
Output mask origin: (-147.8820037841797, 122.4530029296875, 892.5)
Smoothing and resampling time for /mnt/c/users/avery/Desktop/segmentation_masks/patella.nii.gz: 0.98 seconds
Output mask dimensions: (512, 512, 251)
Output mask voxel spacing (mm): (0.48500001430511475, 0.48500001430511475, 1.0)
Output mask origin: (-147.8820037841797, 122.4530029296875, 892.5)
Smoothing and resampling time for /mnt/c/users/avery/Desktop/segmentation_masks/tibia.nii.gz: 1.17 second